Bubble sort proof reference: https://ita.skanev.com/02/problems/02.html

**Claim:** Let $\lt_0$ be a strict linear order of pairs $(i, j)$ of indices $1, \ldots, n$ satisfying the **Singleton Condition**, that is: $i \ne j \implies (i, i) \lt_0 (i, j)$ .  Then there exist sets $S_1, \ldots, S_n$ such that
- $\vert S_i \cup S_j \vert \lt \vert S_k \cup S_l \vert \quad iff \quad (i, j) \lt_0 (k, l)$

***

**Proof:** Let $\lt_0$ be our strict linear order as described.  We construct the sets $S_1, \ldots, S_n$ by the following "bubble-sort-of" algorithm (where $\texttt{S[i]}$ refers to $S_i$, $\texttt{indices}$ is the list containing $1, \ldots, n$, and $\texttt{ordering}$ is a list of pairs representing our ordering $\lt_0$):

In [10]:
def SiConstruction(indices, ordering):
    # S[i] refers to our set Si
    n = len(indices)
    S = [set() for i in range(n)]

    # We keep a counter in order to produce fresh elements whenever we may need them.
    fresh_elem = 1
    
    for (p, q) in ordering[1:len(ordering)]:
        # We iterate backwards through pairs, swapping them as needed.
        for (k, l) in ordering[1:len(ordering)][::-1]:
            # (i, j) is the pair to the left of (k, l)
            (i, j) = ordering[ordering.index((k, l))-1]
            
            if len(S[i].union(S[j])) >= len(S[k].union(S[l])):
                # Swap the two unions.
                
                # First, we populate a list of fresh points to use for pumping.
                pump_count = len(S[i].union(S[j])) - len(S[k].union(S[l])) + 1
                fresh_points = []
                for p in range(pump_count):
                    fresh_points.append(fresh_elem)
                    fresh_elem += 1

                # Then, we pump up every set Sz that is not Si or Sj
                for z in range(n):
                    if (z != i and z != j):
                        S[z].update(fresh_points)

                if pump_count > 0:
                    swapped = True
    
    return S


This algorithm follows the structure of bubblesort; at each stage, while there is still a pair to be swapped, we swap _every_ pair $S_i \cup Sj$, $S_k \cup S_l$ whose sizes are out of order (iterating backwards through the pairs).  In order to actually swap $S_i \cup S_j$ with $S_k \cup S_l$ (where, before swapping, $|S_i \cup S_j| \ge |S_k \cup S_l|$), we add $|S_i \cup S_j| - |S_k \cup S_l|$ fresh points to _every_ set $S_z \ne S_i, S_j$.  This swap ensures that $S_k \cup S_l$ increases in size to be strictly greater than the size of $S_i \cup S_j$, while $S_i \cup S_j$ retains its original size.

Since this swap step affects unions aside from the swapped $S_i \cup S_j$, $S_k \cup S_l$, the cut-and-paste proof that bubblesort works will not suffice for verifying that this algorithm works.  We do take inspiration from this proof, though, and so what follows is a bubblesort-inspired verification:

We must show that $\texttt{SiConstruction}$ **1)** terminates, and **2)** that the resulting $S_i \cup S_j$ (for $i, j \in 1, \ldots n$) satisfy the ordering specified by $\lt_0$, i.e. that 
- $\vert S_i \cup S_j \vert \lt \vert S_k \cup S_l \vert \quad iff \quad (i, j) \lt_0 (k, l)$

***

**1)**

***

**2)** We will describe loop invariants in order to inductively reason about $\texttt{SiConstruction}$.  For convenience, we will call the _union that is currently in the_ $w^{th}$ _position of the list_ $\cup_w$ (whereas the union _involving_ sets $S_i, S_j$ we call $S_i \cup S_j$).

***

**Inner Loop Invariant:** At the start of each inner loop iteration (the $v^{th}$ iteration, starting from $v = n$ and decrementing), $\cup_v$ has strictly smaller size than any of $[\cup_{v+1}, \ldots, \cup_n]$, while the rest get shuffled.

**Proof:** By induction on the number of passes through the inner loop.

**Base Step:** At the start of the first pass through the inner loop, $v = n$, so $\cup_v$ is the last union in the list $\cup_n$.  This means that $[\cup_{v+1}, \ldots, \cup{n}]$ is empty.  So the inner loop invariant holds trivially.

**Inductive Step:** Suppose for induction that at the start of the $v^th$ pass through the inner loop, our loop invariant holds.  Since we are decrementing $v$, we want to show that the invariant holds at the start of the $(v-1)^{st}$ path (rather than the $(v+1)^{st}$ pass).

Since our loop invariant holds at the start of the $v^{th}$ pass, we have $\cup_v$ has strictly smaller size than any of $[\cup_{v+1}, \ldots, \cup{n}]$, whereas the rest get shuffled. 

We have two cases:
1. $|\cup_{v-1}| \lt |\cup_{v}|$
2. $|\cup_{v-1}| \ge |\cup_{v}|$

In case **(1)**, we do not add any fresh elements.  We get that, since $|\cup_{v-1}| \lt |\cup_{v}|$, $\cup_{v-1}$ has strictly smaller size than any of $[\cup_{v}, \cup_{v+1}, \ldots, \cup{n}]$, while the rest get shuffled.

Case **(2)** is a little more tricky, since we don't "just swap" the two unions.  Say $\cup_{v-1}$ is $S_i \cup S_j$ and $\cup_{v}$ is $S_k \cup S_l$, and so $|S_i \cup S_j| \ge |S_k \cup S_l|$, yet $(i, j) \lt_0 (k, l)$ (which is consistent with this point in the code).

We add $|S_i \cup S_j| - |S_k \cup S_l|$ fresh points to _every_ set $S_z \ne S_i, S_j$.  As a result, of course, $|S_i \cup S_j| \lt |S_k \cup S_l|$.  In addition, all of those $\cup_u$ _above_ $S_k \cup S_l$ _before the pumping_ also obtain these new fresh points, since they each involve a set $S_z \ne S_i, S_j$.  (This last claim requires justification.  $\cup_u$ cannot be $S_i \cup S_j$, since this is listed _below_ $S_k \cup S_l$.  In addition, $\cup_u$ cannot be $S_i \cup S_i$ or $S_j \cup S_j$ because our ordering $lt_0$ satisfies the Singleton Condition, which would cause $S_i \cup S_i$ or $S_j \cup S_j$ to sink to the position of $S_k \cup S_l$ (TODO NOT CONVINCING).  So $\cup_u$ must involve some set $S_z \ne S_i, S_j$).  This pumping maintains the ordering of $[S_k \cup S_l, \ldots, \cup_n]$.

It might be the case that these fresh elements result in $\cup_{v-2} \le \cup_{v-1}$, but this is no matter at this stage.  We still get our desired result, i.e.:

- At the start of the $(v-1)^{st}$ loop iteration, $\cup_{v-1}$ has strictly smaller size than any of $[\cup_v, \ldots, \cup_n]$, while the rest get shuffled.

***

**Outer Loop Invariant:** At the start of each outer loop iteration (the $w^{th}$ iteration), we have that:

1. $\vert S_i \cup S_j \vert \lt \vert S_k \cup S_l \vert \quad iff \quad (i, j) \lt_0 (k, l) \quad$ for all $S_i \cup S_j, S_k \cup S_l$ in the initial segment $[\cup_1, \ldots, \cup_{w-1}]$ (i.e. this segment is completely sorted), and

2. For every union $S_i \cup S_j \in [\cup_1, \ldots, \cup_{w-1}]$ and every union $S_p \cup S_q \in [\cup_w, \ldots, \cup_n$], $\quad$ $|S_i \cup S_j| \lt |S_p \cup S_q| iff (i, j) \lt_0 (p, q)$.

**Proof:** TODOFIX By induction on the number of passes through the outer loop.  

**Base Step:** At the start of the first pass through the outer loop, the initial segment is empty.  This is trivially sorted, and its elements are trivially strictly less than those of the final segment.  

**Inductive Step:** Suppose for induction that at the start of the $w^{th}$ pass through the outer loop, our loop invariant holds.

We consider the start of the $(w+1)^{st}$ pass (i.e. after executing the inner loop during the $w^{th}$ pass).

By **(1)**, the initial segment $\cup_1, \ldots, \cup_{w-1}$ is completely sorted (before we do the $(w+1)^{st}$ pass).  After this next pass, we do not swap any of $[\cup_1, \ldots, \cup_{w-1}]$, because each are always less than the element adjacent to them (on the right).  So after this next pass, this initial segment is still completely sorted.  In addition, by **(2)**, $|\cup_w-1| \lt |\cup_w|$.  This means that the initial segment _including_ $w$ is completely sorted, i.e.

- $\vert S_i \cup S_j \vert \lt \vert S_k \cup S_l \vert \quad iff \quad (i, j) \lt_0 (k, l) \quad$ for all $S_i \cup S_j, S_k \cup S_l$ in $[\cup_1, \ldots, \cup_{w-1}, \cup_w]$.

This is the condition **(1)** for the start of the $(w+1)^{st}$ pass.  We will now show condition **(2)** still holds.

After the inner loop of the $w^{th}$ pass, $\cup_w$ has the smallest (TODO: strict?) size of any of the unions in the final segment $\cup_{w+1}, \ldots, S_\psi \cup S_\omega$ (by our Inner Loop Invariant), while the rest of the unions in the final segment are shuffled in size.  In addition, by **(2)** for the $w^{th}$ pass, all of these unions in $[\cup_1, \ldots, \cup_{w-1}]$ have smaller size than those in $[\cup_1, \ldots, \cup_w]$.  So we conclude that:

- All of these unions in $[\cup_1, \ldots, \cup_{w-1}, \cup_w]$ have sizes that are _strictly smaller_ than those in the final segment $[\cup_{w+1}, \ldots, \cup_n]$.

So the Outer Loop Invariant holds at the start of each pass through the loop.

***

Now we can prove that the resulting $S_i \cup S_j$ are sorted according to the ordering $\lt_0$:

**Claim:** For all pairs $S_i \cup S_j, S_k \cup S_l$: 
- $\vert S_i \cup S_j \vert \lt \vert S_k \cup S_l \vert \quad iff \quad (i, j) \lt_0 (k, l)$

**Proof:** By induction on TODO  
**Base Step:**  
**Inductive Step:**

***